In [ ]:
# Install dependencies
!pip install wikipedia-api gensim transformers tensorflow keras numpy

import wikipediaapi
import re
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LSTM, Bidirectional, Add, LayerNormalization, Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW

# Load Wikipedia data
user_agent = "MyWikipediaBot/1.0 (contact: myemail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(language="en", user_agent=user_agent)

# Define function to fetch & clean Wikipedia text
def get_wiki_text(topic):
    page = wiki_wiki.page(topic)
    text = page.text.lower()
    text = re.sub(r'[^a-z ]', '', text)  # Keep only letters & spaces
    return text

# Fetch multiple Wikipedia pages
topics = ["Artificial Intelligence", "Neural Networks", "Machine Learning"]
wiki_clean_text = " ".join([get_wiki_text(topic) for topic in topics])

# Data Augmentation: Add noise & randomly delete characters
def augment_text(text, drop_prob=0.05):
    text = list(text)
    for i in range(len(text)):
        if random.random() < drop_prob:
            text[i] = ''
    return ''.join(text)

wiki_clean_text = augment_text(wiki_clean_text)

# Tokenization
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([wiki_clean_text])

# Convert text to numerical sequences
seq_length = 10
sequences, next_chars = [], []
encoded = tokenizer.texts_to_sequences([wiki_clean_text])[0]

for i in range(len(encoded) - seq_length):
    sequences.append(encoded[i:i+seq_length])
    next_chars.append(encoded[i+seq_length])

X = pad_sequences(sequences, maxlen=seq_length, padding='pre')
y = np.array(next_chars)


In [ ]:
# Install dependencies
!pip install wikipedia-api gensim transformers tensorflow keras numpy

import wikipediaapi
import re
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LSTM, Bidirectional, Add, LayerNormalization, Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW

# Load Wikipedia data
user_agent = "MyWikipediaBot/1.0 (contact: myemail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(language="en", user_agent=user_agent)

# Define function to fetch & clean Wikipedia text
def get_wiki_text(topic):
    page = wiki_wiki.page(topic)
    text = page.text.lower()
    text = re.sub(r'[^a-z ]', '', text)  # Keep only letters & spaces
    return text

# Fetch multiple Wikipedia pages
topics = ["Artificial Intelligence", "Neural Networks", "Machine Learning"]
wiki_clean_text = " ".join([get_wiki_text(topic) for topic in topics])

# Data Augmentation: Add noise & randomly delete characters
def augment_text(text, drop_prob=0.05):
    text = list(text)
    for i in range(len(text)):
        if random.random() < drop_prob:
            text[i] = ''
    return ''.join(text)

wiki_clean_text = augment_text(wiki_clean_text)

# Tokenization
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([wiki_clean_text])

# Convert text to numerical sequences
seq_length = 10
sequences, next_chars = [], []
encoded = tokenizer.texts_to_sequences([wiki_clean_text])[0]

for i in range(len(encoded) - seq_length):
    sequences.append(encoded[i:i+seq_length])
    next_chars.append(encoded[i+seq_length])

X = pad_sequences(sequences, maxlen=seq_length, padding='pre')
y = np.array(next_chars)

# Define Transformer Self-Attention Block
def transformer_block(inputs):
    attn = Attention()([inputs, inputs])  # Self-attention
    attn = Add()([inputs, attn])  # Residual Connection
    norm = LayerNormalization()(attn)  # Layer Normalization
    return norm

# Define Transformer + BiLSTM Model
def build_model(vocab_size, embed_dim, seq_length):
    inputs = Input(shape=(seq_length,))
    embed = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_length)(inputs)

    attn_out = transformer_block(embed)  # Transformer Layer
    lstm_out = Bidirectional(LSTM(256, return_sequences=False))(attn_out)  # BiLSTM
    dropout = Dropout(0.3)(lstm_out)
    outputs = Dense(vocab_size, activation='softmax')(dropout)

    model = Model(inputs, outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Build model
model = build_model(len(tokenizer.word_index) + 1, 300, seq_length)

# Training settings
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)

# Train model
model.fit(X, y, epochs=30, batch_size=64, callbacks=[lr_scheduler, early_stopping])

# Function to Predict Next Character
def predict_next_char(input_text):
    encoded = tokenizer.texts_to_sequences([input_text])
    encoded = pad_sequences([encoded[0][-seq_length:]], maxlen=seq_length, padding='pre')
    prediction = model.predict(encoded, verbose=0)
    predicted_char_index = np.argmax(prediction)

    for char, index in tokenizer.word_index.items():
        if index == predicted_char_index:
            return char
    return ""

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=4858da09e3d58f941d1d6aff92e6a7d4b107cdb6ce86785e71b4454710c8d0ac
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 333s 155ms/step - accuracy: 0.3283 - loss: 2.2770 - learning_rate: 0.0010
Epoch 2/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 346s 164ms/step - accuracy: 0.4554 - loss: 1.8630 - learning_rate: 0.0010
Epoch 3/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 374s 160ms/step - accuracy: 0.4933 - loss: 1.7320 - learning_rate: 0.0010
Epoch 4/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 348s 165ms/step - accuracy: 0.5143 - loss: 1.6556 - learning_rate: 0.0010
Epoch 5/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 379s 164ms/step - accuracy: 0.5335 - loss: 1.5886 - learning_rate: 0.0010
Epoch 6/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 375s 161ms/step - accuracy: 0.5497 - loss: 1.5318 - learning_rate: 0.0010
Epoch 7/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 382s 161ms/step - accuracy: 0.5576 - loss: 1.4933 - learning_rate: 0.0010
Epoch 8/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 405s 172ms/step - accuracy: 0.5755 - loss: 1.4384 - learning_rate: 0.0010
Epoch 9/30
2109/2109 ━━━━━━━━━━━━━━━━━━━━ 366s 164ms/step - accuracy: 0.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
# Real-time User Input Prediction
while True:
    user_input = input("Type something: ")
    if user_input.lower() == "exit":
        break
    lstm_char = predict_next_char(user_input)

    print(f"LSTM Predicted: {lstm_char} ")


Type something: hi


NameError: name 'predict_next_char' is not defined

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Save the trained model in the new format
model.save("/content/drive/MyDrive/bilstm_transformer_model.keras")

# Save the tokenizer separately
import pickle

with open("/content/drive/MyDrive/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print(" Model and tokenizer saved in Google Drive using the recommended format!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import load_model
import pickle
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')
# Load the model from Google Drive
model = load_model("/content/drive/MyDrive/bilstm_transformer_model.keras")

# Load the tokenizer
with open("/content/drive/MyDrive/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

print("Model and tokenizer loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model and tokenizer loaded successfully!


In [ ]:
#  Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#  Step 2: Load the Saved Model
from tensorflow.keras.models import load_model

model_bilstm_transformer = load_model("/content/drive/MyDrive/bilstm_transformer_model.keras")

#  Step 3: Load the Tokenizer
import pickle

with open("/content/drive/MyDrive/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

print(" Model and tokenizer loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model and tokenizer loaded successfully!


In [ ]:
from tensorflow.keras.models import Model

#  Extract BiLSTM hidden states before softmax
feature_extractor = Model(
    inputs=model_bilstm_transformer.input,
    outputs=model_bilstm_transformer.get_layer("bidirectional").output  # Extract BiLSTM hidden states
)

#  Function to get BiLSTM features for any input text
def get_bilstm_features(input_text):
    encoded = tokenizer.texts_to_sequences([input_text])
    encoded = pad_sequences([encoded[0][-seq_length:]], maxlen=seq_length, padding='pre')
    return feature_extractor.predict(encoded)


In [ ]:
import numpy as np

#  Function to Predict Next Character
def predict_next_char(input_text):
    # Extract BiLSTM features
    encoded = tokenizer.texts_to_sequences([input_text])
    encoded = pad_sequences([encoded[0][-seq_length:]], maxlen=seq_length, padding='pre')

    # Predict next character from Transformer + BiLSTM model
    prediction = model_bilstm_transformer.predict(encoded, verbose=0)
    predicted_char_index = np.argmax(prediction)

    # Convert back to character
    for char, index in tokenizer.word_index.items():
        if index == predicted_char_index:
            return char
    return ""

#  Generate Text by Predicting Next Characters
user_input = "artificial inte"
for _ in range(5):  # Generate 5 characters
    next_char = predict_next_char(user_input)
    user_input += next_char
    print(f"Generated Text: {user_input}")


Generated Text: artificial intel
Generated Text: artificial intell
Generated Text: artificial intelli
Generated Text: artificial intellig
Generated Text: artificial intellige


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

#  Function to Predict Next Character
def predict_next_char(input_text):
    encoded = tokenizer.texts_to_sequences([input_text])
    encoded = pad_sequences([encoded[0][-seq_length:]], maxlen=seq_length, padding='pre')

    # Predict next character
    prediction = model_bilstm_transformer.predict(encoded, verbose=0)
    predicted_char_index = np.argmax(prediction)

    # Convert back to character
    for char, index in tokenizer.word_index.items():
        if index == predicted_char_index:
            return char
    return ""  # If no match found

#  Interactive Typing & Prediction Loop
print(" Type a word or sentence and press Enter to predict the next character.")
print(" Type 'exit' to stop the program.\n")

while True:
    user_input = input(" Type something: ").strip()  # Get user input
    if user_input.lower() == "exit":
        print(" Exiting...")
        break  # Exit loop when user types 'exit'

    next_char = predict_next_char(user_input)
    print(f" Predicted Next Character: {next_char}\n")


🔹 Type a word or sentence and press Enter to predict the next character.
🔹 Type 'exit' to stop the program.

📝 Type something: hi my nam
✨ Predicted Next Character: e

📝 Type something: how was y
✨ Predicted Next Character: o

📝 Type something: exit
🚪 Exiting...


## **VAE, is not WORKING!!**

In [ ]:
from tensorflow.keras.models import Model

# Extract LSTM Output Instead of Softmax Predictions
feature_extractor = Model(
    inputs=model_bilstm_transformer.input,
    outputs=model_bilstm_transformer.get_layer("bidirectional").output  # Extract BiLSTM hidden states
)

# Extract hidden state features from the model
X_train_features = feature_extractor.predict(X)  # X is our preprocessed input data
print(f"✅ Extracted Features Shape: {X_train_features.shape}")


4213/4213 ━━━━━━━━━━━━━━━━━━━━ 132s 31ms/step
✅ Extracted Features Shape: (134786, 512)


In [ ]:

# # Print Model Summary to Find Correct Layer Names
# vae.summary()


In [ ]:
# ✅ Extract Features from Pre-trained BiLSTM + Transformer Model
feature_extractor = Model(
    inputs=model_bilstm_transformer.input,
    outputs=model_bilstm_transformer.get_layer("bidirectional").output  # Extract BiLSTM hidden states
)

# ✅ Extract Features for All Training Data Instead of One Sample
X_train_features = feature_extractor.predict(X)  # X is the full dataset

print(f"✅ Extracted Features Shape: {X_train_features.shape}")  # Expected: (Dataset Size, 512)


4220/4220 ━━━━━━━━━━━━━━━━━━━━ 126s 30ms/step
✅ Extracted Features Shape: (135023, 512)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, LayerNormalization, GaussianNoise
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

# ✅ Define Sampling Layer for Variational Autoencoder (VAE)
def sampling(args):
    """Reparameterization trick to sample latent space"""
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))  # Random noise
    return z_mean + K.exp(0.5 * z_log_var) * epsilon  # Reparameterization Trick

# ✅ Reduce Latent Dimension Size
latent_dim = 16

# ✅ Define Input Layer (Takes BiLSTM Features)
encoder_inputs = Input(shape=(512,), name="encoder_input")

# ✅ Add Gaussian Noise to Prevent Overfitting
noisy_inputs = GaussianNoise(0.05, name="noisy_input")(encoder_inputs)

# ✅ Encoder (Variational Bottleneck)
z_mean = Dense(latent_dim, activation="tanh", name="z_mean")(noisy_inputs)
z_log_var = Dense(latent_dim, activation="tanh", name="z_log_var")(noisy_inputs)

# ✅ Prevent KL Collapse by Clipping Log Variance
z_log_var = Lambda(lambda x: K.clip(x, -4, 4))(z_log_var)  # Further constrained

# ✅ Apply Layer Normalization Before Sampling
z_mean = LayerNormalization(name="z_mean_norm")(z_mean)
z_log_var = LayerNormalization(name="z_log_var_norm")(z_log_var)

z = Lambda(sampling, name="sampling_layer")([z_mean, z_log_var])

# ✅ Compute KL Divergence (No Negative Loss)
def kl_loss_layer(args):
    """KL loss inside a Lambda layer (Fix for Negative KL)"""
    z_mean, z_log_var = args
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1, keepdims=True)
    return K.abs(kl_loss)  # Ensure KL loss is always positive

kl_loss_output = Lambda(kl_loss_layer, name="kl_loss")([z_mean, z_log_var])

# ✅ KL Annealing (Gradually Increase KL Loss Weight)
class KLScheduler(tf.keras.callbacks.Callback):
    def __init__(self, max_weight=0.3, increase_per_epoch=0.02):  # Lower max weight
        super(KLScheduler, self).__init__()
        self.max_weight = max_weight
        self.increase_per_epoch = increase_per_epoch
        self.current_weight = 0.01  # Start with very low KL weight

    def on_epoch_end(self, epoch, logs=None):
        if self.current_weight < self.max_weight:
            self.current_weight += self.increase_per_epoch
            print(f"✅ KL Loss Weight Updated: {self.current_weight}")  # Debugging info

# ✅ Add Small Decoder for Latent Space Regularization
decoder_hidden = Dense(32, activation="relu", name="decoder_hidden")(z)
decoder_output = Dense(512, activation="linear", name="decoder_output")(decoder_hidden)

# ✅ Define Full VAE Model (Outputs Latent Variables + KL Loss)
vae = Model(encoder_inputs, [decoder_output, kl_loss_output], name="VAE_LatentExtractor")

# ✅ Define Losses
def reconstruction_loss(y_true, y_pred):
    """Mean squared error for feature reconstruction"""
    return K.mean(K.square(y_true - y_pred))

# ✅ Compile VAE (Fix KL Loss Collapse & NaN Issues)
vae.compile(
    optimizer=Adam(learning_rate=0.00005),  # Further reduce learning rate
    loss={
        "decoder_output": reconstruction_loss,
        "kl_loss": lambda y_true, y_pred: K.mean(y_pred)  # Fix KL loss to prevent NaNs
    }
)

print("✅ VAE Model Compiled Successfully!")

# ✅ **Pre-Training Check for NaN Values in Input**
if np.isnan(X_train_features).any():
    raise ValueError("❌ X_train_features contains NaN values. Check preprocessing!")

# ✅ Train VAE to Extract Latent Features
vae.fit(
    X_train_features,
    {
        "decoder_output": X_train_features,
        "kl_loss": np.zeros((X_train_features.shape[0], 1))
    },
    epochs=10,
    batch_size=64,
    callbacks=[KLScheduler(max_weight=0.3, increase_per_epoch=0.02)]  # KL Annealing
)

print("✅ VAE Training Complete!")

# ✅ Extract VAE-Refined Features
X_train_latent, _ = vae.predict(X_train_features)
print(f"✅ Extracted Latent Features Shape: {X_train_latent.shape}")  # Should be (N, latent_dim)


✅ VAE Model Compiled Successfully!
Epoch 1/10
2101/2107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - decoder_output_loss: 0.1505 - kl_loss_loss: 11.0983 - loss: 11.2488✅ KL Loss Weight Updated: 0.03
2107/2107 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - decoder_output_loss: 0.1504 - kl_loss_loss: 11.0931 - loss: 11.2436
Epoch 2/10
2096/2107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - decoder_output_loss: 0.1112 - kl_loss_loss: 5.9922 - loss: 6.1033✅ KL Loss Weight Updated: 0.05
2107/2107 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - decoder_output_loss: 0.1112 - kl_loss_loss: 5.9878 - loss: 6.0990
Epoch 3/10
2102/2107 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - decoder_output_loss: 0.1091 - kl_loss_loss: 3.3406 - loss: 3.4497✅ KL Loss Weight Updated: 0.07
2107/2107 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - decoder_output_loss: 0.1091 - kl_loss_loss: 3.3393 - loss: 3.4484
Epoch 4/10
2102/2107 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - decoder_output_loss: 0.1087 - kl_loss_loss: 1.7017 - loss: 1.8104✅ KL Loss Weight Updated: 0.09000000000000001
21

In [ ]:
# ✅ Extract latent space (instead of reconstruction)
latent_feature_extractor = Model(inputs=vae.input, outputs=vae.get_layer("sampling_layer").output)

X_train_latent = latent_feature_extractor.predict(X_train_features)

print(f"✅ Extracted Latent Features Shape: {X_train_latent.shape}")
# Expected: (134786, 16) since latent_dim = 16


4213/4213 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
✅ Extracted Latent Features Shape: (134786, 16)


In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # Ensure it includes all possible labels


In [ ]:
print(f"Vocab Size: {vocab_size}, Max Label in y: {np.max(y)}")


Vocab Size: 28, Max Label in y: 27


In [ ]:
y = np.clip(y, 0, vocab_size - 1)  # Ensure all labels are within range


In [ ]:
output = Dense(vocab_size, activation="softmax", name="char_output")(bilstm)


In [ ]:
# ✅ Define `vocab_size` Correctly
vocab_size = len(tokenizer.word_index) + 1  # Ensure correct size

# ✅ Ensure Labels are Within Valid Range
y = np.clip(y, 0, vocab_size - 1)  # Fix out-of-range labels

# ✅ Define Input Layer
vae_input = Input(shape=(X_train_latent.shape[1],), name="vae_input")

# ✅ Expand Dimensions for LSTM
vae_reshaped = Reshape((1, X_train_latent.shape[1]), name="vae_reshaped")(vae_input)

# ✅ Add a Simple BiLSTM Classifier
bilstm = Bidirectional(LSTM(128, return_sequences=False), name="bilstm")(vae_reshaped)
output = Dense(vocab_size, activation="softmax", name="char_output")(bilstm)

# ✅ Define Model
vae_classifier = Model(vae_input, output, name="VAE_Classifier")
vae_classifier.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# ✅ Train Model on VAE Features
vae_classifier.fit(X_train_latent, y, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.1440 - loss: 2.9579 - val_accuracy: 0.1457 - val_loss: 2.8831
Epoch 2/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.1570 - loss: 2.8641 - val_accuracy: 0.1457 - val_loss: 2.8819
Epoch 3/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1525 - loss: 2.8649 - val_accuracy: 0.1457 - val_loss: 2.8834
Epoch 4/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.1539 - loss: 2.8631 - val_accuracy: 0.1457 - val_loss: 2.8857
Epoch 5/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.1563 - loss: 2.8597 - val_accuracy: 0.1456 - val_loss: 2.8849
Epoch 6/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.1536 - loss: 2.8609 - val_accuracy: 0.1456 - val_loss: 2.8852
Epoch 7/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1539 - loss: 2.8559 - val_accuracy: 0.1459 - val_loss: 2.8883
Epoch 8/10
1896/1896 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1549 - loss: 2